# Skriti Markovi modeli

Model zapišemo s slovarjem slovarjev. Na primer, za metanje kovancev:

![Loaded coin model.](figs/07-loaded-coin.jpg)

In [1]:
T = {"F": {"F": 0.95, "L": 0.05},
     "L": {"F": 0.05, "L": 0.95}}
E = {"F": {"o": 0.5, "-": 0.5},
     "L": {"o": 0.7, "-": 0.3}}
start = "F"

## Generiranje zaporedij

<font color="green"><b>Naredi sam/a.</b></font> Zapiši funkcijo generate_hmm_sequence, ki sprejme skriti Markov model in vrne zaporedje dolžine n (skrito in vidno zaporedje).

Še prej zapišite funkcijo weighted_choice, ki na podlagi uteži (v vrednosti) naključno izbere vrednost (v ključu slovarja).

In [2]:
import random
random.seed(42)

def weighted_choice(weighted_items):
    """Random choice given the list of elements and their weights"""
    rnd = random.random() * sum(weighted_items.values())
    for i, w in weighted_items.items():
        rnd -= w
        if rnd < 0:
            return i

Zdaj pa funkcijo generate_hmm_sequence:

In [3]:
def generate_hmm_sequence(h, T, E, n):
    """
    HMM sequence given start state,
    transition, emission matrix and sequence length
    
    return zip(hidden_path, visible_sequence)
    """

    s = weighted_choice(E[h])
    yield h, s
    for _ in range(n-1):
        h = weighted_choice(T[h])
        yield h, weighted_choice(E[h])

Generiraj nekaj zaporedij različnih dolžin.

In [4]:
list(generate_hmm_sequence('F', T, E, 5))

[('F', '-'), ('F', 'o'), ('F', '-'), ('F', '-'), ('F', 'o')]

In [5]:
list(generate_hmm_sequence('F', T, E, 20))

[('F', 'o'),
 ('F', '-'),
 ('F', 'o'),
 ('F', '-'),
 ('F', '-'),
 ('F', 'o'),
 ('F', '-'),
 ('F', 'o'),
 ('L', 'o'),
 ('L', 'o'),
 ('L', 'o'),
 ('L', '-'),
 ('L', '-'),
 ('L', 'o'),
 ('L', 'o'),
 ('L', 'o'),
 ('L', 'o'),
 ('L', 'o'),
 ('L', 'o'),
 ('L', 'o')]

Model poskusite uporabiti tudi na primeru goljufivega kazinoja:

![Loaded dice model.](figs/07-loaded-dice.jpg)

In [6]:
A = A = ["1", "2", "3", "4", "5", "6"]
E = {"F": {a: 1/6. for a in A},
     "L": {a: 1/10 if a != "6" else 0.5 for a in A}}
T = {"F": {"F": 0.95, "L": 0.05},
     "L": {"F": 0.1, "L": 0.9}}
start = "F"

In [7]:
list(generate_hmm_sequence('F', T, E, 5))

[('F', '4'), ('F', '3'), ('F', '2'), ('F', '4'), ('F', '2')]

In [8]:
list(generate_hmm_sequence('F', T, E, 40))

[('F', '5'),
 ('F', '3'),
 ('L', '6'),
 ('L', '6'),
 ('L', '6'),
 ('L', '1'),
 ('L', '3'),
 ('L', '6'),
 ('L', '4'),
 ('L', '4'),
 ('L', '5'),
 ('L', '3'),
 ('L', '3'),
 ('L', '6'),
 ('L', '3'),
 ('L', '6'),
 ('F', '1'),
 ('F', '4'),
 ('F', '3'),
 ('F', '3'),
 ('L', '6'),
 ('L', '6'),
 ('F', '5'),
 ('F', '4'),
 ('F', '4'),
 ('F', '3'),
 ('F', '6'),
 ('F', '2'),
 ('F', '2'),
 ('F', '6'),
 ('F', '4'),
 ('F', '1'),
 ('F', '4'),
 ('F', '4'),
 ('F', '2'),
 ('F', '6'),
 ('F', '5'),
 ('F', '1'),
 ('F', '6'),
 ('F', '3')]

## Gradnja modela iz podatkov

Napišite funkcijo build_hmm, ki bo sprejela vidno in skrito zaporedje, ter vrnila skriti Markov model, slovarja T in E.

In [9]:
from collections import Counter

def normalize(dic):
    s = sum(dic.values())
    return {k: dic[k]/s for k in dic}

def build_hmm(h, x):
    t = {}
    for (i, j), cn in Counter(zip(h, h[1:])).items():
        t.setdefault(i, {}).setdefault(j, cn)
    T = {}
    for i, d in t.items():
        T[i] = normalize(d)
        
    c = Counter(zip(h, x))
    E = {}
    for h in T.keys():
        E[h] = normalize({xi: c[(pi, xi)] for pi, xi in c if pi == h})
    return T, E

In [10]:
h, x = zip(*list(generate_hmm_sequence('F', T, E, 400)))

In [11]:
build_hmm(h, x)

({'F': {'F': 0.9550561797752809, 'L': 0.0449438202247191},
  'L': {'F': 0.09090909090909091, 'L': 0.9090909090909091}},
 {'F': {'1': 0.13805970149253732,
   '2': 0.1865671641791045,
   '3': 0.19029850746268656,
   '4': 0.14925373134328357,
   '5': 0.17537313432835822,
   '6': 0.16044776119402984},
  'L': {'1': 0.08333333333333333,
   '2': 0.08333333333333333,
   '3': 0.10606060606060606,
   '4': 0.06060606060606061,
   '5': 0.10606060606060606,
   '6': 0.5606060606060606}})

## Viterbijev algoritem

Zaporedja, s katerimi delamo, so lahko zelo dolga. Množenje (majhnih) verjetnosti nas lahko hitro privede do napake *underflow*. Težavi se izognemo tako, da namesto množenja verjetnosti, seštevamo logaritme verjetnosti.

In [12]:
import math

def logmv(a):
    min_val = 0.00000000001
    return math.log(max(a, min_val))

def viterbi_log(s, t, e):

    # seznam skritih stanj
    zh = set()
    for h, tmpd in e.items():
        zh.add(h)

    zh = [0] + list(zh)

    # Create table V
    V = [{} for i in range(len(s)+1)]
    ptr = [{} for i in range(len(s)+1)]

    # Initialize i = 0; V(0, 0) = 1; V(k, 0) = 0 for k > 0
    for k in zh:
        V[0][k] = logmv(0.0) #t[0][k]*e[k][s[0]]
    V[0][0] = logmv(1.0)

    # for 1 = 1 : n, compute
    for i in range(1, len(s)+1):
        for l in zh:
            vals = list([(V[i-1].get(k, logmv(0.0))+logmv(t.get(k, {}).get(l, 0.0)), k) for k in zh])
            max_val, max_k = max(vals)
            V[i][l] = logmv(e.get(l, {}).get(s[i-1], 0.0)) + max_val
            ptr[i][l] = max_k

    # trace back
    pi = []
    pi_L = max([(V[-1][k], k) for k in zh])[1]
    pi.append(pi_L)

    for p in ptr[-1:1:-1]:
        pi.append(p[pi[-1]])

    pi.reverse()
    return V, zh, ptr, "".join(pi)

Pokliči funkcijo, ki za dano zaporedje x in za dani model (T in E) vrne najbolj verjetno skrito pot (h_najv).

In [13]:
V, zh, ptr, h_najv = viterbi_log(x, T, E)

In [14]:
h_najv

'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'

Primerjaj jo z dejansko skrito potjo:

In [15]:
"".join(h)

'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLFFFLLLLLLLLLLLLLLFFFFFFFFLLLLFFFFFFFFFFFFFFFFFFFFFFFFLFFFFFFFLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLFFFFFFFLLLLLLLLLLLLFFFFFFFFFFFFFFLLLLLLLFFFFFFFFFFFLLLLLLFFLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'

Poti se ujemata v deležu:

In [16]:
sum(pi == pj for pi, pj in zip(h_najv, h))/len(h)

0.855